**airflow - 실습**. 



Krx 에서 전종목 기본정보와 , 오늘의 공시 정보 함수를 호출하여  ,
전종목 기본정보 업데이트 시작 후 오늘의 공시 정보 업데이트 후 각각 데이터 베이스 적재를 한 뒤
적재된 데이터 끼리 비교하여 해당되는 필드에 값이 있을 경우 업데이트가 될 수 있게끔 자동으로 동작하게 짜보았다.

In [ ]:

# 모듈 설치

import json
import pendulum
import requests
import traceback
from datetime import date, datetime

from airflow import DAG
from airflow.decorators import task
from airflow.exceptions import AirflowFailException

from config.dp import config as conf_dp
from util.proc_log import ProcLogger

local_tz = pendulum.timezone('Asia/Seoul')
now = datetime.now(tz=local_tz)
start_date = datetime(2023, 2, 8, tzinfo=local_tz)

proc_logger = ProcLogger(conf_dp)

default_args = {
  'owner' : 'sjbssk0809',
}



# krx 전종목 기본정보 업데이트 함수 호출


@task(task_id='final_listed_issue')
def final_listed_issue():
  try:
    is_exception = True
    # slack.send('#채널명', 'korea_stock_adj()')

    url = f'{conf_dp.scrap_krx_stock_url}/issue/finalTestAllData'
    
    # task = proc_logger.proc_start(method='POST', target_url=url)

    res = requests.get(url, headers=conf_dp.headers,  timeout=120)
    # res = requests.post(url, headers=conf_dp.headers, data=json.dumps(data), timeout=120)
    # proc_logger.proc_end(task['id'], res.status_code, res.text)

    # if res.status_code < 300:
    #   results = res.json()
    #   return len(results)
    # else:
    #   is_exception = False
    #   raise AirflowFailException(res.text)
  except Exception:
    err_msg = traceback.format_exc()
    # if is_exception:
    #   # slack.send('#채널명', err_msg)
    #   proc_logger.proc_end(task['id'], 600, err_msg)
    # raise AirflowFailException(err_msg)






# krx(kind.krx.com) 오늘의 공시 업데이트 함수 호출



@task(task_id='kind_disclosure')
def kind_disclosure():
  try:
    is_exception = True
    # slack.send('#채널명', 'kospi_index_price ()')

    url = f'{conf_dp.scrap_krx_url}/disclosure/disclosureup'
   
    # task = proc_logger.proc_start(method='POST', target_url=url, param=json.dumps(data))

    res = requests.get(url,   timeout=300)
    # res = requests.post(url, headers=conf_dp.headers, data=json.dumps(data), timeout=120)
    # proc_logger.proc_end(task['id'], res.status_code, res.text)

    # if res.status_code < 300:
    #   return res.text
    # else:
    #   is_exception = False
    #   raise AirflowFailException(res.text)
  except Exception:
    err_msg = traceback.format_exc()
    # if is_exception:
    #   # slack.send('#채널명', err_msg)
    #   proc_logger.proc_end(task['id'], 600, err_msg)
    raise AirflowFailException(err_msg)









# 최종 업데이트 함수 호출




@task(task_id='krxcomparsion_issue_disclosure')
def krxcomparsion_issue_disclosure():
  try:
    is_exception = True

    url = f'{conf_dp.scrap_krx_url}/disclosure/issuedisclosure'

    res = requests.get(url,   timeout=300)

  
  except Exception:
    err_msg = traceback.format_exc()
    # if is_exception:
    #   # slack.send('#채널명', err_msg)
    #   proc_logger.proc_end(task['id'], 600, err_msg)
    raise AirflowFailException(err_msg)



# dag 은 airflow의 흐름을 설명해준다.


with DAG(
  dag_id = 'scrap_krx_stock_issue_disclosure',
  start_date = start_date, 
  schedule_interval = '40 10 * * 1-5',
  catchup = False,
  default_args = default_args,
  tags = ['test'],
) as dag:



# 진행 순서 




  final_listed_issue() >> kind_disclosure() >>krxcomparsion_issue_disclosure()
